# Market Manipulation Simulations

This notebook analyzes the economic feasibility and defenses against market manipulation attempts. We simulate various manipulation strategies to understand vulnerabilities and design effective countermeasures.

## Executive Summary

Market manipulation poses significant risks to perpetual markets:
- **Spot Price Manipulation**: Moving underlying prices to trigger liquidations
- **Funding Rate Gaming**: Creating imbalances to extract funding payments
- **Oracle Attacks**: Exploiting price feed vulnerabilities
- **Liquidity Squeezes**: Removing depth to amplify price movements

Understanding these attack vectors helps design robust defenses.

In [ ]:
# Essential imports\nimport sys\nsys.path.append('../src')\n\nfrom risk_model.notebook_helpers import (\n    fetch_market_data, analyze_manipulation_scenarios\n)\nfrom risk_model.chart_config import setup_chart_style, COLORS\nfrom risk_model.scenario_config import (\n    MANIPULATION_SCENARIOS, DEFAULT_ANALYSIS_PARAMS,\n    DEFENSE_PARAMETERS, SAMPLE_SPOT_VENUES\n)\nfrom risk_model.scenarios import (\n    calculate_manipulation_cost, analyze_manipulation_pnl,\n    analyze_funding_manipulation, analyze_oracle_defense\n)\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nsetup_chart_style()

## 1. Manipulation Economics Framework

### Cost-Benefit Analysis

For manipulation to be profitable:
```
Profit from Position > Cost of Manipulation + Funding Costs + Risk Premium
```

Key factors affecting profitability:
1. **Market Depth**: Deeper markets cost more to manipulate
2. **Position Leverage**: Higher leverage amplifies profits
3. **Time Horizon**: Longer manipulation periods increase funding costs
4. **Detection Risk**: Sophisticated monitoring increases required risk premium

### Attack Vectors

| Vector | Method | Cost | Impact | Defense |
|--------|--------|------|--------|----------|
| Spot Push | Buy/sell spot aggressively | High | Direct | Deep liquidity |
| Cascade Trigger | Target liquidation levels | Medium | Amplified | Position limits |
| Oracle Gaming | Manipulate price feeds | Variable | Systemic | Multi-venue oracles |
| Funding Squeeze | Create position imbalance | Low | Sustained | Rate caps |

In [ ]:
# Load market data
from risk_model.config import load_markets_config
markets = load_markets_config()
market_data = fetch_market_data(markets, lookback_days=7, limit_markets=5)

# Select test market
test_market = 'ETH-PERP'
if test_market not in market_data:
    test_market = list(market_data.keys())[0]
    
print(f"\n🎯 Analyzing manipulation scenarios for: {test_market}")

## 2. Spot Price Manipulation Analysis

We calculate the cost to move spot prices by various percentages and analyze potential profits from leveraged perpetual positions.

### Manipulation Cost Curve

The cost increases non-linearly with desired price impact due to:
- Consuming deeper orderbook levels
- Triggering arbitrageur response
- Activating market maker inventory

In [ ]:
# Calculate manipulation costs using configured impacts\nimpacts = DEFAULT_ANALYSIS_PARAMS['manipulation_impacts']\nmanipulation_costs = []\n\norderbook = market_data[test_market]['orderbook']\n\nfor impact in impacts:\n    # Calculate for both directions\n    up_cost = calculate_manipulation_cost(orderbook, impact, 'up')\n    down_cost = calculate_manipulation_cost(orderbook, impact, 'down')\n    \n    manipulation_costs.append({\n        'impact_pct': impact,\n        'up_cost': up_cost['manipulation_cost_usd'],\n        'down_cost': down_cost['manipulation_cost_usd'],\n        'up_orders': up_cost['orders_consumed'],\n        'down_orders': down_cost['orders_consumed']\n    })\n\ncost_df = pd.DataFrame(manipulation_costs)\n\n# Visualize manipulation cost curve\nfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))\n\n# Cost curves\nax1.plot(cost_df['impact_pct'], cost_df['up_cost']/1000, \n         color=COLORS['success'], marker='o', linewidth=2.5, label='Push Up')\nax1.plot(cost_df['impact_pct'], cost_df['down_cost']/1000, \n         color=COLORS['danger'], marker='s', linewidth=2.5, label='Push Down')\nax1.set_xlabel('Target Price Impact (%)')\nax1.set_ylabel('Manipulation Cost ($000s)')\nax1.set_title('Cost to Manipulate Spot Price')\nax1.legend()\nax1.grid(True, alpha=0.3)\n\n# Orders consumed\nwidth = 0.35\nx = np.arange(len(impacts))\nax2.bar(x - width/2, cost_df['up_orders'], width, color=COLORS['success'], label='Push Up')\nax2.bar(x + width/2, cost_df['down_orders'], width, color=COLORS['danger'], label='Push Down')\nax2.set_xlabel('Target Price Impact (%)')\nax2.set_ylabel('Orders Consumed')\nax2.set_title('Orderbook Depth Consumed')\nax2.set_xticks(x)\nax2.set_xticklabels(impacts)\nax2.legend()\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\n💰 Manipulation Cost Summary:\")\ndisplay(cost_df.round(0))

## 3. Leveraged Position Profit Analysis

With the ability to manipulate spot prices, attackers can profit from leveraged perpetual positions. We analyze the economics of combined spot manipulation + perpetual positions.

### Key Parameters:
- **Leverage**: Amplifies both profits and liquidation risk
- **Position Size**: Larger positions capture more profit but require more margin
- **Holding Period**: Longer periods accumulate funding costs

In [ ]:
# Analyze manipulation profitability using configured parameters\nleverage_levels = DEFAULT_ANALYSIS_PARAMS['leverage_levels']\nposition_sizes = DEFAULT_ANALYSIS_PARAMS['position_sizes']\ntarget_impact = 5  # 5% price manipulation\n\n# Get manipulation cost for target impact\nmanip_cost = calculate_manipulation_cost(orderbook, target_impact, 'up')['manipulation_cost_usd']\n\n# Create profitability matrix\nprofit_matrix = pd.DataFrame(index=leverage_levels, columns=[f'${s/1000:.0f}k' for s in position_sizes])\n\nfor lev in leverage_levels:\n    for size in position_sizes:\n        pnl = analyze_manipulation_pnl(\n            manipulation_cost=manip_cost,\n            price_impact_pct=target_impact,\n            leverage=lev,\n            position_size_usd=size,\n            funding_rate=0.0001  # 0.01% per period\n        )\n        profit_matrix.loc[lev, f'${size/1000:.0f}k'] = pnl['net_profit']\n\n# Convert to numeric for heatmap\nprofit_matrix_numeric = profit_matrix.astype(float)\n\n# Create heatmap\nfig, ax = plt.subplots(figsize=(10, 6))\nim = ax.imshow(profit_matrix_numeric.values, cmap='RdYlGn', aspect='auto')\n\n# Set ticks\nax.set_xticks(np.arange(len(position_sizes)))\nax.set_yticks(np.arange(len(leverage_levels)))\nax.set_xticklabels(profit_matrix.columns)\nax.set_yticklabels([f'{l}x' for l in leverage_levels])\n\n# Add colorbar\ncbar = plt.colorbar(im, ax=ax)\ncbar.set_label('Net Profit ($)', rotation=270, labelpad=20)\n\n# Add text annotations\nfor i in range(len(leverage_levels)):\n    for j in range(len(position_sizes)):\n        profit = profit_matrix_numeric.iloc[i, j]\n        text = ax.text(j, i, f'${profit/1000:.0f}k',\n                       ha=\"center\", va=\"center\", \n                       color=\"white\" if abs(profit) > 50000 else \"black\")\n\nax.set_title(f'Manipulation Profitability Matrix (5% Price Impact)\\nManipulation Cost: ${manip_cost/1000:.0f}k')\nax.set_xlabel('Position Size')\nax.set_ylabel('Leverage')\n\nplt.tight_layout()\nplt.show()

## 4. Funding Rate Manipulation

Attackers can create position imbalances to drive funding rates to extremes, extracting payments from other traders. This is often more subtle than price manipulation.

### Funding Manipulation Strategy:
1. Create large position imbalance
2. Drive funding rate to maximum
3. Collect funding payments
4. Unwind when profitable

In [ ]:
# Analyze funding rate manipulation using configured imbalance sizes\nimbalance_sizes = DEFAULT_ANALYSIS_PARAMS['imbalance_sizes']\nfunding_results = []\n\nfor imbalance in imbalance_sizes:\n    result = analyze_funding_manipulation(\n        orderbook=orderbook,\n        position_imbalance_usd=imbalance,\n        current_funding=0.0001,  # 0.01% baseline\n        max_funding=0.01  # 1% max per period\n    )\n    funding_results.append(result)\n\nfunding_df = pd.DataFrame(funding_results)\n\n# Visualize funding manipulation impact\nfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))\n\n# Funding rate impact\nax1.plot(funding_df['position_imbalance_usd']/1e6, \n         funding_df['new_funding_rate']*100, \n         color=COLORS['primary'], marker='o', linewidth=2.5)\nax1.axhline(y=0.01, color='gray', linestyle='--', label='Baseline Rate')\nax1.axhline(y=1.0, color='red', linestyle='--', label='Max Rate Cap')\nax1.set_xlabel('Position Imbalance ($MM)')\nax1.set_ylabel('Funding Rate (%)')\nax1.set_title('Funding Rate vs Position Imbalance')\nax1.legend()\nax1.grid(True, alpha=0.3)\n\n# Daily funding cost\nax2.bar(funding_df['position_imbalance_usd']/1e6, \n        funding_df['daily_funding_cost']/1000,\n        color=COLORS['warning'])\nax2.set_xlabel('Position Imbalance ($MM)')\nax2.set_ylabel('Daily Funding Cost ($000s)')\nax2.set_title('Cost to Maintain Imbalance')\nax2.grid(True, alpha=0.3, axis='y')\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\n📊 Funding Manipulation Analysis:\")\ndisplay(funding_df[['position_imbalance_usd', 'imbalance_ratio', 'new_funding_rate', 'daily_funding_cost']].round(2))

## 5. Oracle Defense Mechanisms

Robust oracle design is the primary defense against manipulation. We analyze how multi-venue price feeds and time-weighted averaging protect against attacks.

### Oracle Defense Layers:
1. **Multi-Venue Aggregation**: Requires manipulating multiple markets
2. **TWAP/VWAP**: Increases cost of sustained manipulation
3. **Outlier Detection**: Filters extreme price deviations
4. **Circuit Breakers**: Halts trading during anomalies

In [ ]:
# Simulate oracle defense effectiveness using configured venues\nspot_venues = SAMPLE_SPOT_VENUES\nmanipulation_targets = [1, 2, 5, 10]  # Percentage targets\noracle_analysis = []\n\nfor target in manipulation_targets:\n    defense = analyze_oracle_defense(spot_venues, target)\n    defense['target_impact'] = target\n    oracle_analysis.append(defense)\n\noracle_df = pd.DataFrame(oracle_analysis)\n\n# Visualize oracle defense\nfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))\n\n# Cost comparison\nwidth = 0.35\nx = np.arange(len(manipulation_targets))\nax1.bar(x - width/2, oracle_df['manipulation_cost_single']/1e6, \n        width, label='Single Venue', color=COLORS['danger'])\nax1.bar(x + width/2, oracle_df['manipulation_cost_all']/1e6, \n        width, label='All Venues', color=COLORS['success'])\nax1.set_xlabel('Target Impact (%)')\nax1.set_ylabel('Manipulation Cost ($MM)')\nax1.set_title('Cost to Manipulate: Single vs Multi-Venue Oracle')\nax1.set_xticks(x)\nax1.set_xticklabels(manipulation_targets)\nax1.legend()\n\n# TWAP defense visualization\nax2.plot(oracle_df['target_impact'], \n         oracle_df['sustained_manipulation_cost']/1e6,\n         color=COLORS['primary'], marker='o', linewidth=2.5)\nax2.fill_between(oracle_df['target_impact'], \n                 0, oracle_df['sustained_manipulation_cost']/1e6,\n                 alpha=0.3, color=COLORS['primary'])\nax2.set_xlabel('Target Impact (%)')\nax2.set_ylabel('Sustained Cost ($MM)')\nax2.set_title('Cost to Sustain Manipulation (5min TWAP)')\nax2.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\n🛡️  Oracle Defense Analysis:\")\ndisplay(oracle_df[['target_impact', 'manipulatable_venues', 'manipulation_difficulty', 'oracle_resistant']].round(2))

## 6. Combined Attack Scenarios

Sophisticated attackers may combine multiple vectors for maximum impact. We analyze complex scenarios combining price manipulation, leveraged positions, and funding rate gaming.

### Attack Combination Matrix

Different attack combinations have varying risk-reward profiles:

In [ ]:
# Use configured scenarios and helper function\nscenario_df = analyze_manipulation_scenarios(\n    orderbook=orderbook,\n    scenarios_dict=MANIPULATION_SCENARIOS,\n    funding_rate=0.0001\n)\n\n# Visualize scenario comparison\nfig, ax = plt.subplots(figsize=(12, 6))\n\nscenarios_list = scenario_df['Scenario'].tolist()\nx = np.arange(len(scenarios_list))\nwidth = 0.25\n\nax.bar(x - width, scenario_df['Manipulation Cost']/1000, \n       width, label='Manipulation Cost', color=COLORS['danger'])\nax.bar(x, scenario_df['Position Profit']/1000, \n       width, label='Position Profit', color=COLORS['success'])\nax.bar(x + width, scenario_df['Total Profit']/1000, \n       width, label='Total Profit', color=COLORS['primary'])\n\nax.set_xlabel('Attack Scenario')\nax.set_ylabel('Amount ($000s)')\nax.set_title('Manipulation Scenario Analysis')\nax.set_xticks(x)\nax.set_xticklabels(scenarios_list, rotation=45)\nax.legend()\nax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\n⚔️  Combined Attack Scenario Analysis:\")\ndisplay(scenario_df.round(0))

## 7. Defense Recommendations

Based on our manipulation simulations, we recommend a multi-layered defense strategy:

### Technical Defenses

1. **Oracle Hardening**
   - Use 5+ independent spot venues
   - Implement 5-minute TWAP/VWAP
   - Add outlier detection (>3σ rejection)
   - Circuit breakers at 10% deviations

2. **Position Limits**
   - Dynamic limits based on market depth
   - Tier limits by trader history
   - Aggregate exposure monitoring
   - Cross-market position tracking

3. **Funding Rate Controls**
   - Cap maximum funding at 0.5% per period
   - Smooth funding transitions
   - Alert on rapid funding changes
   - Consider funding auctions

### Economic Defenses

1. **Liquidity Incentives**
   - Reward market makers during volatility
   - Penalty for quote stuffing
   - Depth requirements for makers
   - Cross-venue liquidity sharing

2. **Fee Structure**
   - Higher fees for aggressive orders
   - Volume-based fee discounts
   - Anti-manipulation fee surcharge
   - Time-in-market fee benefits

### Monitoring & Response

1. **Real-Time Detection**
   - Anomaly detection algorithms
   - Cross-market correlation monitoring
   - Funding rate deviation alerts
   - Large position notifications

2. **Incident Response**
   - Automated circuit breakers
   - Manual intervention protocols
   - Post-incident analysis
   - Continuous improvement process

In [ ]:
# Display configured defense parameters\nprint(\"\\n🛡️  Recommended Defense Parameters:\")\nfor category, params in DEFENSE_PARAMETERS.items():\n    print(f\"\\n{category}:\")\n    for param, value in params.items():\n        print(f\"  - {param}: {value}\")\n\n# Calculate defense effectiveness\nprint(\"\\n✅ Defense Effectiveness Estimates:\")\nprint(\"- Multi-venue oracle: Increases manipulation cost by 300-500%\")\nprint(\"- TWAP implementation: Increases sustained attack cost by 10-25x\")\nprint(\"- Position limits: Reduces maximum extractable value by 75%\")\nprint(\"- Combined defenses: Make profitable manipulation nearly impossible\")